# Import thư viện

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import traceback

In [2]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument("--start-maximized")
s = Service('C:\chromedriver')
driver = webdriver.Chrome(service=s, options=options)
driver.implicitly_wait(0)

In [3]:
# List tất cả url movie
list_all_url_movie = pd.read_csv('E:/Crawl_Web/Rotten Tomatoes/Crawl URL/URL_movie.csv').squeeze('columns').tolist()

In [4]:
df_movie = []

for url_movie in list_all_url_movie[:5]:
    # Mở url phim
    driver.get(url_movie)
    
    try:
        # Đợi cho đến khi url load xong
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="main_container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
        soup = BeautifulSoup(html_of_interest, 'lxml')

        sleep(2)

        ## Tạo Dict chứa thông tin về phim
        dict_info_movie = {}

        # Tiêu đề phim
        dict_info_movie['Title'] = soup.select('h1[class="scoreboard__title"]')[0].text

        # Điểm đánh giá của nhà phê bình
        dict_info_movie['Tomatometer score'] = soup.select('score-board')[0].attrs['tomatometerscore']

        # Điểm đánh giá của khán giả
        dict_info_movie['Audience score'] = soup.select('score-board')[0].attrs['audiencescore']

        # Số lượt đánh giá của nhà phê bình
        dict_info_movie['Tomatometer count'] = soup.select('a[slot="critics-count"]')[0].text.strip()

        # Số lượt đánh giá của khán giả
        dict_info_movie['Audience count'] = soup.select('a[slot="audience-count"]')[0].text.strip()

        # Trạng thái đánh giá phim của nhà phê bình
        dict_info_movie['Tomatometer state'] = soup.select('score-board')[0].attrs['tomatometerstate']

        # Trạng thái đánh giá phim của khán giả
        dict_info_movie['Audience state'] = soup.select('score-board')[0].attrs['audiencestate']

        # Thông tin phim
        list_name_info = soup.select('div[class="meta-label subtle"]')
        list_info = soup.select('div[data-qa="movie-info-item-value"]')

        for num_info in range(len(list_name_info)):
            name_info = list_name_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
            info = list_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
            dict_info_movie[name_info] = info

        # Tóm tắt nội dụng phim
        dict_info_movie['Synopsis'] = soup.select('div[id="movieSynopsis"]')[0].text.strip()

        # Diễn viên
        list_raw_cast = soup.select('span[class="characters subtle smaller"]')
        list_cast = []

        for c in list_raw_cast:
            list_cast.append(c.attrs['title'])

        dict_info_movie['Cast'] = ', '.join(list_cast)

        df_movie.append(dict_info_movie)

        # Reviews của nhà phê bình
        driver.get(url_movie + '/reviews')

        element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="layout reviews-page-container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
        soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

        # Check có review của nhà phê bình không ?
        check_critic_review = soup_critic_review.select('div[class="review_table"]')[0].text.strip()

        if check_critic_review != '':
            list_critic_review = []
            list_top_critic_review = []
            
            # Số lượng review trong 1 trang
            num_rv = len(soup_critic_review.select('div[class="review-row"]'))

            for num in range(num_rv):
                rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text
                list_critic_review.append(rv)

                # Check top critic reviews
                check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

                if check_top_critic != []:
                    list_top_critic_review.append(rv)
            
            while True:
                sleep(1)
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
                    driver.execute_script("arguments[0].click();", next_button)
                    
                    element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="review_table"]')))
                    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
                    soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

                    num_rv = len(soup_critic_review.select('div[class="review-row"]'))

                    for num in range(num_rv):
                        rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text.strip()
                        list_critic_review.append(rv)

                        check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

                        if check_top_critic != []:
                            list_top_critic_review.append(rv)
                    
                except:
                    break
            dict_info_movie['Critic review'] = '<>'.join(list_critic_review)
            if list_top_critic_review != []:
                dict_info_movie['Top critic review'] = '<>'.join(list_top_critic_review)
            else:
                dict_info_movie['Top critic review'] = 'No top critic reviews'
        elif check_critic_review == '':
            dict_info_movie['Critic review'] = 'No critic reviews'
            dict_info_movie['Top critic review'] = 'No top critic reviews'

        # Reviews của khán giả
        driver.get(url_movie + '/reviews?type=user')

        element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-qa="reviews-container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
        soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

        ## Check có review của khán giả không ?
        check_audience_review = soup_audience_review.select('div[id="movieUserReviewsContent"]')

        if len(check_audience_review) != 0:
            list_audience_review = []
            list_verified_audience_review = []
            
            # Số lượng review trong 1 trang
            num_audience_rv = len(soup_audience_review.select('div[class="audience-review-row"]'))
            
            for num in range(num_audience_rv):
                rv = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('p', {'data-qa': 'review-text'})[0].text
                list_audience_review.append(rv)

                # Check verified audience review
                check_verified_audience = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('span', {'class': 'audience-reviews__verified'})

                if check_verified_audience != []:
                    list_verified_audience_review.append(rv)
            
            while True:
                sleep(1)
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
                    driver.execute_script("arguments[0].click();", next_button)
                    
                    element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="movieUserReviewsContent"]')))
                    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
                    soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

                    # Số lượng review trong 1 trang
                    num_audience_rv = len(soup_audience_review.select('div[class="audience-review-row"]'))
                    
                    for num in range(num_audience_rv):
                        rv = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('p', {'data-qa': 'review-text'})[0].text
                        list_audience_review.append(rv)

                        # Check verified audience review
                        check_verified_audience = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('span', {'class': 'audience-reviews__verified'})

                        if check_verified_audience != []:
                            list_verified_audience_review.append(rv)
                    
                except:
                    break
            dict_info_movie['Audience review'] = '<>'.join(list_audience_review)
            if list_verified_audience_review != []:
                dict_info_movie['Verified audience review'] = '<>'.join(list_verified_audience_review)
            else:
                dict_info_movie['Verified audience review'] = 'No verified audience reviews'

        else:
            dict_info_movie['Audience review'] = 'No audience reviews'
            dict_info_movie['Verified audience review'] = 'No verified audience reviews'

    except Exception:
        traceback.print_exc()
        print(url_movie)
        continue
    
df_movie = pd.DataFrame(df_movie)

In [7]:
df_movie

,Title,Tomatometer score,Audience score,Tomatometer count,Audience count,Tomatometer state,Audience state,Genre,Original Language,Director,...,Synopsis,Cast,Critic review,Top critic review,Audience review,Verified audience review,Producer,Release Date (Theaters),Box Office (Gross USA),Distributor
0,Beg!,,,0 Reviews,50+ Ratings,,,Horror,English,Robert Golden,...,Murder and mayhem unfold as two rivals jockey ...,"Peta Lily, Philip Pellew, Julian Bleach, Olega...",No critic reviews,No top critic reviews,I'm not really sure what to say about this. I...,No verified audience reviews,NaN,NaN,NaN,NaN
1,Limo,,,0 Reviews,0 Ratings,,,Adventure,Dutch,Guy Goossens,...,Four girls visit a mysterious village.,"Jan Decleir, Jack Wouterse, Jaap Spijkers, Ing...",No critic reviews,No top critic reviews,No audience reviews,No verified audience reviews,"Guy Goossens, Mark Punt",NaN,NaN,NaN
2,Tejano,,71,4 Reviews,Fewer than 50 Ratings,,upright,Mystery & thriller,English,David Blue Garcia,...,A farmhand in South Texas breaks his arm to sm...,"Patrick Mackie, Roland Uribe, Mayra Leal, Adri...",The story of a South Texas ranch hand who gets...,An entertaining blast of Texas noir that nods ...,The movie is a good western and David is a goo...,No verified audience reviews,David Blue Garcia,NaN,NaN,NaN
3,Outpatient,,,0 Reviews,0 Ratings,,,Drama,English,Alec Carlen,...,A doctor (Catherine Kellner) and a publisher t...,"Justin Kirk, Catherine Kellner, Claudia Mason,...",No critic reviews,No top critic reviews,No audience reviews,No verified audience reviews,NaN,NaN,NaN,NaN
4,Paa,50,67,8 Reviews,"1,000+ Ratings",rotten,upright,Drama,Hindi,R. Balki,...,A politician's 13-year-old son has a rare diso...,"Amitabh Bachchan, Abhishek Bachchan, Vidya Bal...","The film, which peters off into vague sub-plot...","Disappointingly, Paa is not as out-of-the-box ...",What a refreshing experience<>Fantastic movie....,No verified audience reviews,Sunil Manchanda,"Dec 4, 2009 limited",$199.2K,Big Pictures


In [6]:
# df_movie.to_csv('Crawl_test_1000_movie.csv', index=False)